In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2023-07-29 09:07:19--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 64.233.170.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  19.9MB/s    in 4.0s    

2023-07-29 09:07:23 (16.5 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import zipfile
import os
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 59s 469ms/step - loss: 1.5619 - acc: 0.6175 - val_loss: 0.4201 - val_acc: 0.8150
Epoch 2/10
100/100 [==============================] - 37s 371ms/step - loss: 0.5136 - acc: 0.7430 - val_loss: 0.3131 - val_acc: 0.8680
Epoch 3/10
100/100 [==============================] - 37s 366ms/step - loss: 0.4570 - acc: 0.7880 - val_loss: 0.3049 - val_acc: 0.8640
Epoch 4/10
100/100 [==============================] - 36s 359ms/step - loss: 0.4221 - acc: 0.8090 - val_loss: 0.2536 - val_acc: 0.8980
Epoch 5/10
100/100 [==============================] - 36s 359ms/step - loss: 0.3802 - acc: 0.8260 - val_loss: 0.2753 - val_acc: 0.8780
Epoch 6/10
100/100 [==============================] - 39s 386ms/step - loss: 0.3876 - acc: 0.8255 - val_loss: 0.2170 - val_acc: 0.9110
Epoch 7/10
100/100 [==============================] - 36s 361ms/step - loss: 0.3890 - acc: 0.8285 - val_loss: 0.2102 - val_acc: 0.9160
Epoch 8/10
100/100 [==============================] - 3

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (150, 150))
validation_generator = test_datagen.flow_from_directory(validation_dir, batch_size = 20, class_mode = 'binary', target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10


<ipython-input-17-8d91753fef64>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


100/100 [==============================] - 26s 200ms/step - loss: 10.8356 - acc: 0.8315 - val_loss: 0.2139 - val_acc: 0.9600
Epoch 2/10
100/100 [==============================] - 19s 185ms/step - loss: 0.5140 - acc: 0.8770 - val_loss: 0.1795 - val_acc: 0.9480
Epoch 3/10
100/100 [==============================] - 18s 182ms/step - loss: 0.3229 - acc: 0.8920 - val_loss: 0.1338 - val_acc: 0.9490
Epoch 4/10
100/100 [==============================] - 18s 177ms/step - loss: 0.2763 - acc: 0.9015 - val_loss: 0.1419 - val_acc: 0.9560
Epoch 5/10
100/100 [==============================] - 19s 186ms/step - loss: 0.2479 - acc: 0.9055 - val_loss: 0.3389 - val_acc: 0.9150
Epoch 6/10
100/100 [==============================] - 23s 227ms/step - loss: 0.2317 - acc: 0.9170 - val_loss: 0.1027 - val_acc: 0.9660
Epoch 7/10
100/100 [==============================] - 17s 172ms/step - loss: 0.2196 - acc: 0.9210 - val_loss: 0.1169 - val_acc: 0.9420
Epoch 8/10
100/100 [==============================] - 17s 175ms/s

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( validation_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [ ]:
for layers in base_model.layers:
  layers.trainable=False

In [ ]:
from tensorflow.python.keras.layers.core import Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))

for layers in base_model.layers:
  layers.trainable=False

base_model.add(Dense(500, activation='relu'))
base_model.add(Dropout(0.2))
base_model.add(Dense(500, activation='relu'))
base_model.add(Dropout(0.2))
base_model.add(Dense(1, activation='sigmoid'))

In [ ]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(lr=0.001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
early_stopping =tf.keras.callbacks.EarlyStopping(patience=20)

In [ ]:
resnet_history = model.fit(train_generator, validation_data = validation_generator, epochs = 100,callbacks=[early_stopping])


Epoch 1/100
100/100 [==============================] - 34s 337ms/step - loss: 0.6655 - acc: 0.5975 - val_loss: 0.6454 - val_acc: 0.6240
Epoch 2/100
100/100 [==============================] - 33s 325ms/step - loss: 0.6603 - acc: 0.6025 - val_loss: 0.6491 - val_acc: 0.6230
Epoch 3/100
100/100 [==============================] - 37s 371ms/step - loss: 0.6611 - acc: 0.6055 - val_loss: 0.6625 - val_acc: 0.5980
Epoch 4/100
100/100 [==============================] - 33s 328ms/step - loss: 0.6586 - acc: 0.6080 - val_loss: 0.6435 - val_acc: 0.6280
Epoch 5/100
100/100 [==============================] - 34s 340ms/step - loss: 0.6614 - acc: 0.5985 - val_loss: 0.6434 - val_acc: 0.6310
Epoch 6/100
100/100 [==============================] - 34s 341ms/step - loss: 0.6582 - acc: 0.6135 - val_loss: 0.6431 - val_acc: 0.6190
Epoch 7/100
100/100 [==============================] - 33s 327ms/step - loss: 0.6578 - acc: 0.6065 - val_loss: 0.6444 - val_acc: 0.6230
Epoch 8/100
100/100 [===========================